<a href="https://colab.research.google.com/github/JhonatanPeixoto/JhonatanPeixoto/blob/main/Projeto_Final(Allan%2C_Fabio%2C_Jhonatan%2C_Vitor)_Estudo_de_Caso_TripAdvisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

!apt-get update
!apt install chromium-chromedriver

!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, 
            '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
# Importar biblioteca de regex
import re

# Importar biblioteca nltk para remoção de stop words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Biblioteca para remoção de caracteres especiais
from unicodedata import normalize
from string import punctuation

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
browser = webdriver.Chrome('chromedriver', options=chrome_options)

In [ ]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303324-Goiania_State_of_Goias.html'

In [ ]:
browser.get(url)

In [ ]:
b = bs(browser.page_source)

In [ ]:
rests = b.find('div', attrs={'data-test-target':'restaurants-list'})

In [ ]:
class_alvo = rests.find('div').find('div')['class'][0]

In [ ]:
df = pd.DataFrame(columns=['nome', 'avaliacao', 'descricao'])

In [ ]:
linha = 0
for rest in rests.find_all('div', attrs={'class':class_alvo}):
    titulo = rest.find_all('a')[1].text
    avaliacao  = rest.find_all('a')[2].text
    desc   = rest.find_all('span')[14].text
    
    if re.search('\$', desc):
        desc   = rest.find_all('span')[12].text

    df.loc[linha] = [titulo, avaliacao, desc]
    linha+=1

###tratar coluna nome:remover numeração com regex, 
###tratar coluna avaliação: remover o nome "avaliações", converter para int
###tratar coluna descrição: caixa baixa, remover preposições, acentuações, etc


In [ ]:
stop_words = stopwords.words('portuguese')

In [ ]:
stop_words

In [ ]:
def remover_sw(texto):
    words_token = word_tokenize(texto)
    
    # sentenca_limpa = []
    # for w in words_token:
    #     if not w.lower() in stop_words:
    #         sentenca_limpa.append(w)

    sentenca_limpa = [w for w in words_token if not w.lower() in stop_words]

    return ' '.join(sentenca_limpa)

In [ ]:
def limpar_texto(texto):
    # Tratando caracteres especiais
    texto = normalize('NFKD', texto). encode('ASCII', 'ignore').decode('ASCII')
    # Removendo pontuações
    texto = re.sub(f'[{punctuation}]', '', texto)
    # Removendo número
    texto = re.sub('[0-9]', '', texto)
    # Colocar em caixa baixa
    texto = texto.lower()
    # Removendo stop words
    texto = remover_sw(texto)

    return texto

In [ ]:
%%time
df['descricao'] = df['descricao'].map(limpar_texto)

CPU times: user 10.4 ms, sys: 3.87 ms, total: 14.3 ms
Wall time: 15.7 ms


In [ ]:
df

,nome,avaliacao,descricao
0,1. Coco Bambu Flamboyant,4.983 avaliações,brasileira frutos mar
1,2. Taji Japanese Cuisine,827 avaliações,japonesa asiatica
2,3. El Argentino,333 avaliações,steakhouse argentina
3,4. Íz Restaurante,872 avaliações,francesa brasileira
4,5. Ciao Ciao Cucina,176 avaliações,italiana bar
5,6. Restaurante Popular,311 avaliações,brasileira
6,7. Carne de Sol 1008,2.340 avaliações,steakhouse brasileira
7,8. Madero,1.140 avaliações,americana steakhouse
8,9. Pizzaria Pitigliano,721 avaliações,italiana pizza
9,10. Panela Mágica,208 avaliações,brasileira internacional


In [ ]:
rests.find_all('div', attrs={'class':class_alvo})[14]

In [ ]:
for i, s in enumerate(rests.find_all('div', attrs={'class':class_alvo})[14].find_all('span')):
    print(i, s.text)